In [1]:
%connect COM10 --baudrate=115200 --user='micro' --password='python' --wait=0

Connected on COM10


In [1]:
# train controller for set up 1 - circular track around center object - doubles tie fidelity
import hub
import time
from utime import sleep_ms, ticks_ms, ticks_diff

colorSensor = hub.port.D.device
beeper = hub.sound
beeper.volume(2)
trainMotor = hub.port.B
colorSensor.mode(5)

threshold = 2000
tieCounter = 0.0
totalTracks = 16
totalTies = 4 * totalTracks
motorSpeed = 25  # default 35 - lower for shorter train momvement

checkpoints = 16
actionLength = 6
bufferLength = 5  # 2 is absolute min (if train doesn't have to move)

cycleLength = actionLength + bufferLength
tiesPerCheckpoint = int(round(totalTies / checkpoints * 2)) / 2  # rounds to nearest .5


def moveTrain(tiesPerCheckpoint, tieCounter):
    def getTieByRGB():
        colorSensor.mode(5)
        color = colorSensor.get()
        colorValues = (color[0] + color[1] + color[2])
        tie = (colorValues < threshold)
        return tie
    def getTieByColor():
        colorSensor.mode(0)
        color = colorSensor.get()
        tie = (color == [10])
        return tie

    pastTie = getTieByColor()
    trainMotor.pwm(motorSpeed)
    currTies = 0.0

#     print(tieCounter)
    while (tieCounter % tiesPerCheckpoint != 0 or currTies == 0):
        tie = getTieByColor()
        if tie != pastTie:
            tieCounter += .5
            currTies += .5
#             beeper.beep(1000, 50, 0)
#             print(tieCounter)

        pastTie = tie

    trainMotor.pwm(0)
    return tieCounter


def getSeconds():
    seconds = round(ticks_diff(ticks_ms(), start_ms)/1000)
    return seconds


print("ready")
print("start smartphone video")
print("then simultaneously press the right arrow buttons on both spikes")

while not hub.button.right.is_pressed():
    sleep_ms(10)
    
sleep_ms(1500)
print("starting")
start_ms = ticks_ms()


for video in range(checkpoints):
    while (getSeconds() % cycleLength) != 0:
        sleep_ms(10)
    sleep_ms(1000)

    print("recording video " + str(video + 1) + " ... ", end="")
    sleep_ms(1000 * (actionLength))
    print("done")

    sleep_ms(1000)
    tieCounter = moveTrain(tiesPerCheckpoint, tieCounter)

print("recordings finished!")
print("stop smartphone video")


ready
start smartphone video
then simultaneously press the right arrow buttons on both spikes
starting
recording video 1 ... done
recording video 2 ... done
recording video 3 ... done
recording video 4 ... done
recording video 5 ... done
recording video 6 ... done
recording video 7 ... done
recording video 8 ... done
recording video 9 ... done
recording video 10 ... done
recording video 11 ... done
recording video 12 ... done
recording video 13 ... done
recording video 14 ... done
recording video 15 ... done
recording video 16 ... done
recordings finished!
stop smartphone video


In [1]:
# start and stop train
import hub
import time
from utime import sleep_ms

# while not hub.button.right.is_pressed():
#     sleep_ms(50)
    
print("starting")
     
trainMotor = hub.port.B
trainMotor.pwm(30)
time.sleep(1)
trainMotor.pwm(0)


starting


In [1]:
# tie counter checker - now with doubled fidelity
import hub
import time
from utime import sleep_ms

beeper = hub.sound
beeper.volume(2)
colorSensor = hub.port.D.device
colorSensor.mode(5)
threshold = 2000
pastTie = True
tieCounter = 0.0
totalTracks = 16
totalTies = 4 * totalTracks

trainMotor = hub.port.B
trainMotor.pwm(25)

def getTieByRGB():
    colorSensor.mode(5)
    color = colorSensor.get()
    colorValues = (color[0] + color[1] + color[2])
    tie = (colorValues < threshold)
    return tie

def getTieByColor():
    colorSensor.mode(0)
    color = colorSensor.get()
#     print(color)
    tie = (color == [10])
    return tie

while True:

    tie = getTieByColor()
#     print(tie)
    if tie != pastTie:
        tieCounter += .5
        print(tieCounter)
        beeper.beep(1000, 50, 0)
    
    if tieCounter == 64:
        trainMotor.pwm(0)

    pastTie = tie

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256


*** Sending Ctrl-C



In [ ]:
# time checker
from utime import sleep_ms, ticks_ms
import time
timeCompensation = 1.12 # increase if train is ahead of robot
timeCompensation = 1 # increase if train is ahead of robot

def getSeconds():
    seconds = round((ticks_ms()/1000 - startSeconds) * timeCompensation)
    return seconds
    
startSeconds = ticks_ms()/1000
print(ticks_ms())

prev = startSeconds
seconds = getSeconds()
while seconds != 30:
    seconds = getSeconds()
    if seconds != prev:
        print(seconds)
    sleep_ms(50)
    prev = seconds

print("done")

In [ ]:
# train controller for set up 2 - straight track with rotating camera
import hub
import time
from utime import sleep_ms, ticks_ms, ticks_diff

colorSensor = hub.port.D.device
beeper = hub.sound
trainMotor = hub.port.B
trainMotor.pwm(0)

cameraMotor = hub.port.F.motor
colorSensor.mode(5)

beeper = hub.sound
beeper.volume(10)
freq = 2093

threshold = 2000
tieCounter = 0
tracksToTravel = 16 # the number of tracks the train should go forward by 
                    # (note: this should be less than the total number of tracks)
tiesToTravel = 4 * tracksToTravel
angles = 24

degreesPerCheckpoint = int(round(360 / angles))
secondsPerTie = 10 / 64

# videoLength = round(secondsPerTie * tiesToTravel) 
videoLength = 10
# bufferLength = 6 + 3
bufferLength = 15
print("video length: " + str(videoLength))
print("buffer length: " + str(bufferLength))

cycleLength = videoLength + bufferLength


def moveTrain(goalTies, motorPWM):
    def getTieByRGB():
        colorSensor.mode(5)
        color = colorSensor.get()
        colorValues = (color[0] + color[1] + color[2])
        tie = (colorValues < threshold)
        return tie
    def getTieByColor():
        colorSensor.mode(0)
        color = colorSensor.get()
        tie = (color == [10])
        return tie

    pastTie = getTieByColor()
    trainMotor.pwm(motorPWM)
    currTies = 0

#     print(tieCounter)
    while (currTies < goalTies):
        tie = getTieByColor()
        if tie and not pastTie:
            currTies += 1
#             beeper.beep(1000, 50, 0)  # for debug
#             if(currTies % 4 == 0):
#                 print(currTies)
        pastTie = tie
        if (motorPWM < 0 and currTies/goalTies > .70):
            trainMotor.pwm(-25)
        if (currTies/goalTies > .9):
            trainMotor.pwm(slowPWM * int(motorPWM/abs(motorPWM)))
    
    trainMotor.pwm(0)


def getSeconds():
    seconds = round(ticks_diff(ticks_ms(), start_ms)/1000)

    return seconds


print("ready")
print("start smartphone video")
print("then press right arrow button")

while not hub.button.right.is_pressed():
    sleep_ms(10)
    
cameraMotor.preset(cameraMotor.get()[0])
    
sleep_ms(1500)
print("starting")
start_ms = ticks_ms()
therePWM = 32
backPWM = -65
# backPWM = therePWM * -1
slowPWM = 20


for video in range(angles):
    while (getSeconds() % cycleLength) != 0:
        sleep_ms(10)
    beeper.beep(freq, 100, 0)
    sleep_ms(1000)
    
    print("recording video " + str(video + 1) + " ... ", end="")
    moveTrain(tiesToTravel, therePWM)
    print("done")

    sleep_ms(500)
    moveTrain(tiesToTravel, backPWM)
    cameraMotor.run_to_position((video+1)*degreesPerCheckpoint * -1, speed=25)


print("recordings finished!")
print("stop smartphone video")
